# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b 
    ,businesses_version old 
    ,businesses_version new 
where f.filing_type in ('alteration', 'changeOfName')   
    and f.business_id=b.id
    and f.transaction_id=old.end_transaction_id 
    and f.transaction_id=new.transaction_id
    and f.source='LEAR' 
    and old.legal_name != new.legal_name
    and f.id not in (select filing_id from sent_to_gazette);  

In [ ]:
%%sql  coop_gazette_changeofname  <<
select
  to_char(f.effective_date,'MON dd, yyyy')
||';'
|| CASE WHEN b.legal_type='CP' THEN 'CP'||substr(b.identifier,6,4) ||'    '
        WHEN b.legal_type='XCP' THEN 'XCP'||substr(b.identifier,7,4)||'   '
        ELSE b.identifier||' '
   END  
||';'
||rpad(substr(old.legal_name,1,52),52)
||';'
||' to '
||';'
||rpad(substr(new.legal_name,1,58),58)
from filings      f
    ,businesses   b   
    ,sent_to_gazette stg
    ,businesses_version old 
    ,businesses_version new 
where f.filing_type in ('alteration', 'changeOfName')
    and f.business_id=b.id
    and f.id=stg.filing_id  
    and f.transaction_id=old.end_transaction_id 
    and f.transaction_id=new.transaction_id
    and old.legal_name != new.legal_name
    and stg.sent_to_gazette_date is null   
order by f.effective_date;

In [ ]:
datestr = datetime.strftime(datetime.now(), '%Y%m%d')
coop_gazette_changeofname_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_CHANGEOFNAME_' + datestr +'.TXT'

with open(coop_gazette_changeofname_filename, 'w') as f:
    if not coop_gazette_changeofname.DataFrame().empty:
        dfAsString = coop_gazette_changeofname.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b    
where f.filing_type in ('dissolved', 'dissolution') 
    and f.business_id=b.id 
    and f.source='LEAR' 
    and f.id not in (select filing_id from sent_to_gazette);

In [ ]:
%%sql coop_gazette_dissolution  <<
select
  to_char(f.effective_date,'yyyymmdd')
||';'
|| CASE WHEN b.legal_type='CP' THEN 'CP '||substr(b.identifier,3,7)
        WHEN b.legal_type='XCP' THEN 'XCP'||substr(b.identifier,4,7) 
        ELSE 'BC '||substr(b.identifier,3,7)   
   END                     
||';' 
||rpad(b.legal_name,150)
||';'
|| CASE WHEN f.filing_type = 'dissolved' THEN '1'
        WHEN f.filing_type = 'dissolution' THEN '2'        
   end   
||rpad(' ',27)
from filings      f
    ,businesses   b   
    ,sent_to_gazette stg
where f.filing_type in ('dissolved', 'dissolution')  
      and f.business_id=b.id
      and f.id=stg.filing_id      
      and stg.sent_to_gazette_date is null   
order by f.effective_date;      

In [ ]:
coop_gazette_dissolution_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_DISSOLUTION_' + datestr +'.TXT'

with open(coop_gazette_dissolution_filename, 'w') as f:  
    if not coop_gazette_dissolution.DataFrame().empty:
        dfAsString = coop_gazette_dissolution.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b    
where f.filing_type in ('incorporationApplication','amalgamationApplication') 
    and f.business_id=b.id 
    and f.source='LEAR' 
    and f.id not in (select filing_id from sent_to_gazette);

In [ ]:
%%sql coop_gazette_incorporation  <<
select
  to_char(b.founding_date,'MON dd, yyyy')
||';'
|| CASE WHEN b.legal_type='CP' THEN 'CP' ||substr(b.identifier,6,4)||'    '
       WHEN b.legal_type='XCP' THEN 'XCP'||substr(b.identifier,7,4)||'   '
        ELSE b.identifier ||' '
   END                 
||';'
||rpad(b.legal_name,150)
||rpad(' ',59)
from filings      f
    ,businesses   b    
    ,sent_to_gazette stg
where f.filing_type in ('incorporationApplication','amalgamationApplication') 
    and f.business_id=b.id 
    and f.id=stg.filing_id
    and stg.sent_to_gazette_date is null   
order by b.founding_date;

In [ ]:
coop_gazette_incorporation_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_INCORPORATION_' + datestr +'.TXT'

with open(coop_gazette_incorporation_filename, 'w') as f:
    if not coop_gazette_incorporation.DataFrame().empty:
        dfAsString = coop_gazette_incorporation.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b    
where f.filing_type = 'restoration' 
    and f.business_id=b.id 
    and f.source='LEAR' 
    and f.id not in (select filing_id from sent_to_gazette);

In [ ]:
%%sql  coop_gazette_restoration  <<
select
  to_char(f.effective_date,'yyyymmdd')
||';'
|| CASE WHEN b.legal_type='CP' THEN 'CP    '||substr(b.identifier,6,4)
       WHEN b.legal_type='XCP' THEN 'XCP   '||substr(b.identifier,7,4)
       ELSE 'BC '||substr(b.identifier,3,7)
   END                     
||';'
||rpad(b.legal_name,112)
from filings      f
    ,businesses   b    
    ,sent_to_gazette stg
where f.filing_type = 'restoration'  
  and f.business_id=b.id 
  and f.id=stg.filing_id
  and stg.sent_to_gazette_date is null 
order by f.effective_date;

In [ ]:
coop_gazette_restoration_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_RESTORATION_' + datestr +'.TXT'

with open(coop_gazette_restoration_filename, 'w') as f:
    if not coop_gazette_restoration.DataFrame().empty:
        dfAsString = coop_gazette_restoration.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)